In [1]:
import numpy as np
import pandas as pd
import re
from collections import Counter

In [2]:
df = pd.read_csv("../urap/Demo1Evi-2019-06-22T0023-DataHuntAnswers.csv")
backup_df = df
# schema = pd.read_csv("../urap/Demo1Evi-2019-04-17T2143-Schema.csv")
# backup_schema = schema

In [3]:
# Take in the feed, get the contributor_uuid and their answers for each question
def getContributorAndAns(df):
    cols = df.columns
    Q_s = [col_name for col_name in cols if re.search(r"\.Q[0-9]", col_name)]
    selected_cols = ['contributor_uuid'] + Q_s
    return df[selected_cols]
    
df = getContributorAndAns(df)

df.head()

,contributor_uuid,T1.Q2.A1,T1.Q2.A2,T1.Q2.A3,T1.Q3.A1,T1.Q3.A2,T1.Q3.A3,T1.Q3.A4,T1.Q3.A5,T1.Q3.A6,...,T1.Q6,T1.Q7,T1.Q8,T1.Q9,T1.Q11,T1.Q12,T1.Q13,T1.Q14,T1.Q15,T1.Q16
0,e1ae8875-a398-4dde-8f4e-4b21109784e3,1,0,0,1,1,0,0,0,0,...,NaN,NaN,They are not saying that greater surface area ...,Somewhat less likely,Yes,Very Unlikely,Somewhat Unlikely,"Yes, expicitly",5: Middling difficulty,8
1,2abbe1a3-c9a7-41ab-9738-64ca86756d37,1,0,0,1,1,0,1,1,0,...,Can't tell; not enough info,NaN,small sample size,Somewhat less likely,Yes,Somewhat Unlikely,Somewhat Unlikely,"Yes, expicitly",3,7
2,e4a2f20b-0ee3-4d4c-a622-e392d1150ec8,0,0,1,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,No,NaN,NaN,"Yes, implicitly",2,10: 100% certain about all my answers
3,f9143626-bfe0-4e69-b652-6d1525ab4eb0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,No,NaN,NaN,"No, not at all",4,2
4,6ae640df-8dbc-4401-ae14-0636d2c0d086,0,0,1,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,Yes,Somewhat Unlikely,Not Sure,"Yes, expicitly",4,8


for data:

- data is OHE - need to consolidate into one column per question

- rename column titles


# p = 0.01

99% of the time the majority reaches "correct" consensus 

minimum % agreement for majority? bootstrap sampling

*weight using reputation scores (higher scores = better) 

for purposes of this code, we will use 0 to 1*


# variables

n = number of answer choices for questions

k = max number sampled

c = consensus of entire dataset for question

i = sample size

_____
questions = list of questions

n = num answers

answers = answer column

In [4]:
def getConsensus(answers):
    qcounts = Counter(answers)
#     c = list(qcounts.keys())[list(qcounts.values()).index(max(qcounts.values()))]
    maxKey = max(qcounts.keys(), key=lambda key: qcounts[key])
    return maxKey

In [5]:
def strToInt(lst):
    vals = lst.unique()
    mapper = {vals[i]:i for i in range(len(vals))}
    return lst.replace(mapper)

In [12]:
def getN(questionName):
    answers = df[questionName].dropna()
    answers = strToInt(answers)
    n = len(pd.unique(answers))
    c = getConsensus(answers)
    
    for i in range(n + 1, max_group_size):
        count = 0
        for s in range(0, 1000):
            sample = np.random.choice(answers, i, replace=True)
            consensus = getConsensus(sample)
            if consensus == c:
                count += 1
        
        if count/1000 > .99:
            return i
            
    return "Not converged"

In [13]:
#results = pd.DataFrame(columns = ['Question', 'MinUsersForConsensus']) 
cols = ['Question', 'Min']
lst=[]
max_group_size = 100 # CHANGE THIS

for q in df.columns[1:]:
    n = getN(q)
    lst.append([q,n])
    print("Question:", q, "Converge:", n)
    
df1 = pd.DataFrame(lst, columns=cols)

0      0
1      0
2      1
3      1
4      1
      ..
146    0
147    0
148    0
149    1
150    1
Name: T1.Q2.A1, Length: 151, dtype: int64
Question: T1.Q2.A1 Converge: Not converged
0      0
1      0
2      0
3      0
4      0
      ..
146    0
147    0
148    0
149    0
150    0
Name: T1.Q2.A2, Length: 151, dtype: int64
Question: T1.Q2.A2 Converge: 17
0      0
1      0
2      1
3      0
4      1
      ..
146    0
147    0
148    0
149    1
150    1
Name: T1.Q2.A3, Length: 151, dtype: int64
Question: T1.Q2.A3 Converge: 49
0      0
1      0
2      1
3      1
4      1
      ..
146    1
147    0
148    0
149    1
150    1
Name: T1.Q3.A1, Length: 151, dtype: int64
Question: T1.Q3.A1 Converge: 68
0      0
1      0
2      1
3      1
4      1
      ..
146    1
147    1
148    1
149    1
150    1
Name: T1.Q3.A2, Length: 151, dtype: int64
Question: T1.Q3.A2 Converge: 7
0      0
1      0
2      0
3      0
4      0
      ..
146    0
147    0
148    0
149    0
150    0
Name: T1.Q3.A3, Length: 15

KeyboardInterrupt: 